In [1]:
import nsepy
from nsepy import get_history, get_index_pe_history
from datetime import date, datetime , timedelta
import pandas as pd
import random
import os
import ntpath
import glob
import time
import talib
import  warnings
from ipywidgets import IntProgress
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [2]:
data_path =  os.path.join(os.getcwd(),"nse_data\\data\\")
data_index = os.path.join(os.getcwd(),"nse_data\\index_data\\")
dump_path =  os.path.join(os.getcwd(),"nse_data\\dump\\")
meta_data =  os.path.join(os.getcwd(),"nse_data\\meta_data\\")
reports_path = os.path.join(os.getcwd(),"nse_data\\reports\\")
nse_indices = os.path.join(os.getcwd(),"nse_data\\nse_indices\\")
# print(nse_indices)

In [3]:
nifty100_df = pd.read_csv(os.path.join(nse_indices, "ind_nifty100list.csv"))
nifty500_df = pd.read_csv(os.path.join(nse_indices, "ind_nifty500list.csv"))
stocks_list = nifty500_df.Symbol.tolist()
len(stocks_list)

501

In [4]:
stocks_list = list(dict.fromkeys(stocks_list))

add_stocks = ['VEDL', 'YESBANK']
skip_stocks = ['BBTC','IRB','M&MFIN','PVR', 'HATHWAY']

stocks_list = stocks_list + add_stocks

stocks_list = sorted(set(stocks_list)-(set(skip_stocks)))
len(stocks_list)

# stocks_list = random.sample(stocks_list, len(stocks_list))


498

In [5]:
def get_server_last_update_date():
    date_for_update = get_history('TCS', date.today()-timedelta(days=10), date.today()+timedelta(days=1), index = False)
    date_for_update.reset_index(inplace = True)
    return date_for_update.Date.max()

server_date = get_server_last_update_date()
print("Server updated till: ",server_date)

start_date = date(2000, 1, 1)
end_date = date.today()

# start_date = date(2011, 5, 5)
# end_date = date(2011, 5, 9)



Server updated till:  2021-05-21


In [6]:
# Function to get script data file path if present
def get_stock_file_path(script):
    for file in glob.glob(data_path + script + '_*.csv'):
        file = file.replace("\\","/")
        return(file)
    
def get_stock_filename(path):
    head, tail = ntpath.split(path)
    return tail or ntpath.basename(head)
    
# Function to check if the script data file already present or not: True or False
def stock_data_present(script):
    file = get_stock_file_path(script)
    if file:
        return True
    return False

# Function to get stock dataframe from script name
def stock_df_from_csv(script):
    file = get_stock_file_path(script)
    stock_df = pd.read_csv(file)
    return stock_df
 
# Function to get stock dataframe from script name with Date as Index
def get_df_date_index(stock):
    df = stock_df_from_csv(stock)
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', drop=True, inplace = True)
    df.index.name = None
    return df


def calculate_sma(df , t = 20 , field = 'Close'):
    sma = talib.SMA(df[field], timeperiod = t)
    sma.name = 'sma_'+ field +"_"+ str(t)
    sma = sma.to_frame()
    df = df.merge(sma, left_index=True, right_index=True)
    return df

In [7]:
df = get_df_date_index("TCS")
# month_start_date = date.today()
# print(month_start_date.month)


last_updated_date = df.index.max()
last_updated_date

Timestamp('2021-05-21 00:00:00')

In [8]:
# Range Bound
days_backtest = 22
percent_range = 10

range_bound = []
max_count = len(stocks_list)
f = IntProgress(min=0, max=max_count, description='Range Bound:')
display(f)

for stock in stocks_list:
    f.value += 1
    stock_df = stock_df_from_csv(stock)
    stock_df = stock_df.tail(days_backtest)
    high = stock_df.High.max()
    low = stock_df.Low.min()
    close = stock_df.tail(1).Close.values[0]
    if (high-low)/high <= (percent_range/100):
        range_bound.append({'Stock':stock,
                           'High': high,
                           'Low':  low,
                           'Close':close,
                            'Gap':(high-low)*100/high
                           })
range_bound = pd.DataFrame(range_bound)
range_bound = range_bound.sort_values('Gap')        
range_bound = range_bound.reset_index(drop = True)
# range_bound

IntProgress(value=0, description='Range Bound:', max=498)

In [9]:
# 52 week breakout

year_breakout = []

max_count = len(stocks_list)
f = IntProgress(min=0, max=max_count, description='52 Week Breakout:')
display(f)

for stock in stocks_list:
    f.value += 1
    stock_df = stock_df_from_csv(stock)    
    stock_df = stock_df.tail(250)
    
    low = stock_df.iloc[0:-1].Low.min()   
    high = stock_df.iloc[0:-1].High.max()    
    close = stock_df.tail(1).Close.values[0]
    
    if close>high:
        year_breakout.append({'Stock':stock,
                           'Price': high,                           
                           'Close':close,
                            'Break':"Up"
                           })
    
    if close<low:
        year_breakout.append({'Stock':stock,
                           'Price': low,                           
                           'Close':close,
                            'Break':"Down"
                           })
        
year_breakout = pd.DataFrame(year_breakout)
# year_breakout

IntProgress(value=0, description='52 Week Breakout:', max=498)

In [10]:
# Regression Line Trends
max_count = len(stocks_list)
f = IntProgress(min=0, max=max_count, description='Regression Line Trend:')
display(f)

regression_line_trend = []
for stock in stocks_list:
    f.value += 1
    stock_df = get_df_date_index(stock)   
    stock_df = stock_df.tail(100)
    stock_df = calculate_sma(stock_df, 20, field = 'High')
    stock_df = calculate_sma(stock_df, 20, field = 'Low')
    stock_df = stock_df.tail(51)
    stock_df= stock_df.rename(columns={'Prev Close': 'Prev_Close', 'Deliverable Volume': 'Deliverable_Volume'})

    stock_df['sma_avg'] = (stock_df['sma_High_20']+stock_df['sma_Low_20'])/2
    stock_df['sma_avg_high_shift'] = (stock_df['sma_High_20'].shift(periods = 1) +stock_df['sma_Low_20'])/2 
    stock_df['avg_diff'] = stock_df['sma_avg']-stock_df['sma_avg_high_shift']
    stock_df = stock_df.tail(50)
    stock_df = np.round(stock_df, decimals=4)
    reg_line_max_value = stock_df.avg_diff.max()
    reg_line_min_value = stock_df.avg_diff.min()
    
    if(reg_line_max_value < 0 ):
        regression_line_trend.append({'Stock':stock,                           
                                      'Trend': "Falling Regression",
                                      'Regress_Value': reg_line_max_value
                                      })
    if(reg_line_min_value > 0 ):
        regression_line_trend.append({'Stock':stock,                           
                                      'Trend': "Rising Regression",
                                      'Regress_Value': reg_line_min_value
                                      })

regression_line_trend = pd.DataFrame(regression_line_trend)
regression_line_trend = regression_line_trend.sort_values('Regress_Value', ascending = False) 
regression_line_trend = regression_line_trend.reset_index(drop = True)

IntProgress(value=0, description='Regression Line Trend:', max=498)

In [11]:
# Uptrend / Downtrend
period = 10

max_count = len(stocks_list)
f = IntProgress(min=0, max=max_count, description='UP/Down Trend:')
display(f)

trend = []
for stock in stocks_list:
    f.value += 1
    stock_df = stock_df_from_csv(stock)    
    stock_df = stock_df.tail(period*3)
    
    high_1 = stock_df.iloc[-period:].High.max()   
    high_2 = stock_df.iloc[-period*2:-period].High.max()    
    high_3 = stock_df.iloc[-period*3:-period*2].High.max() 
    
    low_1 = stock_df.iloc[-period:].Low.min()   
    low_2 = stock_df.iloc[-period*2:-period].Low.min()  
    low_3 = stock_df.iloc[-period*3:-period*2].Low.min()
    
    if high_1 < high_2 and high_2 < high_3:
        trend.append({'Stock':stock,                           
                      'Trend':"Downtrend"
                      })
    
    if low_1 > low_2 and low_2 > low_3:
        trend.append({'Stock':stock,                         
                      'Trend':"Uptrend"
                      })
        
trend = pd.DataFrame(trend)
print(trend.shape)
# trend.head(10)

IntProgress(value=0, description='UP/Down Trend:', max=498)

(395, 2)


In [12]:
# Uptrend / Downtrend Breakout

trend_breakout = []

max_count = len(stocks_list)
f = IntProgress(min=0, max=max_count, description='UP/Down Trend Breakout:')
display(f)

for stock in stocks_list:
    f.value += 1
    stock_df = stock_df_from_csv(stock)    
    stock_df = stock_df.tail(period*3+1)
    
    close = stock_df.tail(1).Close.values[0]
    high_1 = stock_df.iloc[-period-1:-1].High.max()   
    high_2 = stock_df.iloc[-period*2-1:-period-1].High.max()    
    high_3 = stock_df.iloc[-period*3-1:-period*2-1].High.max() 
     
    low_1 = stock_df.iloc[-period-1:-1].Low.min()   
    low_2 = stock_df.iloc[-period*2-1:-period-1].Low.min()  
    low_3 = stock_df.iloc[-period*3-1:-period*2-1].Low.min()
    
    if high_1 < high_2 and high_2 < high_3 and close > high_1 :
        trend_breakout.append({'Stock':stock,                           
                      'Breakout':"Downtrend Breakout"
                      })
    
    if low_1 > low_2 and low_2 > low_3 and close < low_1 :
        trend_breakout.append({'Stock':stock,                         
                      'Breakout': "Uptrend Breakout"
                      })
        
trend_breakout = pd.DataFrame(trend_breakout)
print(trend_breakout.shape)
# trend_breakout.head(10)

IntProgress(value=0, description='UP/Down Trend Breakout:', max=498)

(6, 2)


In [13]:
trend = pd.DataFrame.merge(trend,trend_breakout, how = 'outer', on = "Stock")
# trend.to_csv(reports_path+"trend.csv", index = True)

In [14]:
datetime_obj = last_updated_date.to_pydatetime().date() 
trend_file = reports_path + 'Trend_Report_' + datetime_obj.strftime('%d-%m-%Y') + '.xlsx'


with pd.ExcelWriter(trend_file) as writer:  
    range_bound.to_excel(writer, sheet_name='Range_Bound_Stocks')
    regression_line_trend.to_excel(writer, sheet_name='Regression Trend')
    year_breakout.to_excel(writer, sheet_name='365 Days Breakout')
    trend.to_excel(writer, sheet_name='Trend Breakout')

print(trend_file, ' Saved to Disk')

C:\Users\Anirudh\Desktop\Technical_Analysis\nse_data\reports\Trend_Report_21-05-2021.xlsx  Saved to Disk
